In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import re

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

USER = "aacuser"
PASS = "simplepass"

# Connect to database via CRUD Module
db = AnimalShelter("aacuser", "simplepass")

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# More debugging
#results = db.read({})
#print("Total records:", len(results))
#print("First 5 records:", results[:5])

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True, errors='ignore')

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height='100px'),
    html.H1("SNHU CS-340 Dashboard"),
    html.H6("Dashboard by Jaden Williams"),
    html.Hr(),

    html.Label("Select Rescue Type:"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',
        labelStyle={'display': 'inline-block', 'margin': '10px'}
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    ),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        row_selectable='single',
    ),

    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water':
        labrador_regex = re.compile(".*labrador retriever mix.*", re.IGNORECASE)
        chesap_regex = re.compile(".*chesa.*", re.IGNORECASE)
        newfound_regex = re.compile(".*newfound.*", re.IGNORECASE)

        query = {
            "$and": [
                {
                    "$or": [
                        { "breed": labrador_regex },
                        { "breed": chesap_regex },
                        { "breed": newfound_regex }
                    ]
                },
                { "sex_upon_outcome": "Intact Female" },
                { "age_upon_outcome_in_weeks": { "$gte": 26.0, "$lte": 156.0 } }
            ]
        }
    elif filter_type == 'mountain':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {}

    # DEBUG:
    #print(f"Query: {query}")
          
    results = db.read(query)
          
    if not results:  # Handle case when no results match
        return [{"breed":"No data available","sex_upon_outcome":"No data","age_upon_outcome_in_weeks":"No data"}]
    
    df_filtered = pd.DataFrame.from_records(results)
    df_filtered.drop(columns=['_id'], inplace=True, errors='ignore')
    
    return df_filtered.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if not viewData:
        return []
    dff = pd.DataFrame(viewData)
    if 'breed' not in dff.columns:
        return []
    return [dcc.Graph(figure=px.pie(dff, names='breed', title='Breed Distribution'))]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData or not index or len(index) == 0:
        return []  # If no data or no selection, just return empty

    dff = pd.DataFrame.from_dict(viewData)
    
    # Ensure the index is valid
    row = index[0] if len(index) > 0 else None
    
    if row is None:
        return []  # If no valid row, return empty

    # Ensure there are valid coordinates in the expected columns
    try:
        lat, lon = dff.iloc[row, 13], dff.iloc[row, 14]
    except IndexError:
        return []  # In case of an invalid index, return empty

    # If we have valid coordinates, render the map
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row, 4]),  # breed
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row, 9])])  # name
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:25980/
